In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
try:
    import tensorflow.compat.v2 as tf
except Exception:
    pass

##tf.enable_v2_behaborior()

from tensorflow.compat.v2 import keras
from tensorflow.compat.v2.keras import layers

print(tf.__version__)

1.14.0


In [5]:
train_data = pd.read_csv('datasets/train.csv')
test_data = pd.read_csv('datasets/test.csv')
data_all = pd.concat([train_data, test_data], ignore_index=True, sort=False)

In [6]:
print(train_data.shape, test_data.shape, data_all.shape)

(116058, 29) (35866, 28) (151924, 29)


In [20]:
data_all.head(10)

,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,4.250,214000,360,2012-03-01,05/2012,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
1,4.875,144000,360,2012-01-01,03/2012,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1.0
2,3.250,366000,180,2012-01-01,03/2012,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
3,4.750,135000,360,2012-02-01,04/2012,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1.0
4,4.750,124000,360,2012-02-01,04/2012,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1.0
5,4.375,150000,360,2012-02-01,04/2012,80,1.0,46.0,675.0,C86,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0
6,4.000,59000,360,2012-02-01,04/2012,95,1.0,44.0,723.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
7,4.500,319000,300,2012-01-01,03/2012,62,1.0,45.0,652.0,A23,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,1.0
8,4.000,520000,360,2012-03-01,05/2012,76,1.0,35.0,808.0,C86,0.0,0.0,0.0,0,0,0,0,1,0,1,0,1,2,0,1,1.0
9,4.000,214000,360,2012-01-01,03/2012,95,2.0,41.0,702.0,A23,30.0,700.0,0.0,0,0,0,0,0,1,1,1,1,1,2,2,1.0


In [22]:
data_all.tail()

,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
151919,4.125,232000,240,01/01/12,Apr-12,80,2.0,30.0,684.0,C86,0.0,712.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
151920,3.375,204000,180,01/01/12,Mar-12,80,1.0,30.0,812.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
151921,4.250,200000,360,01/02/12,Apr-12,52,2.0,38.0,624.0,B12,0.0,646.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,NaN
151922,4.375,400000,360,01/02/12,Apr-12,66,1.0,34.0,753.0,A23,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
151923,4.375,182000,360,01/02/12,Apr-12,70,1.0,3.0,0.0,C86,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [23]:
data_all['total_score'] = data_all['borrower_credit_score'] + data_all['co-borrower_credit_score']
data_all.head()

,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,total_score
0,4.250,214000,360,2012-03-01,05/2012,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1.0,694.0
1,4.875,144000,360,2012-01-01,03/2012,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1.0,697.0
2,3.250,366000,180,2012-01-01,03/2012,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,780.0
3,4.750,135000,360,2012-02-01,04/2012,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1.0,1271.0
4,4.750,124000,360,2012-02-01,04/2012,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1.0,681.0


In [24]:
data_all['insurance_type'].value_counts()

0.0    151432
1.0       492
Name: insurance_type, dtype: int64

In [ ]:
data_all['prem_paid_bro'] = data_all.loc[data_all['insurance_type']==0.0, 'prem_paid_bro'] = 0
data_all['prem_paid_lend'] = data_all.loc[data_all['insurance_type']==1.0, 'prem_paid_bro'] = 1